In [1]:
import os
%pwd

'/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/HIV_inhibitors_classification_and_generation/research'

In [2]:
os.chdir("../")
%pwd

'/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/HIV_inhibitors_classification_and_generation'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path

In [4]:
from hivclass.constants import *
from hivclass.utils.main_utils import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH,
        schema_file_path = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)
        
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file
        )
        
        return data_ingestion_config

In [5]:
import os
import urllib.request as request
from hivclass import logger
from hivclass.utils.main_utils import get_size

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with folowing info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

In [6]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()

except Exception as e:
    raise e

[2025-02-24 23:39:22,316: INFO: main_utils: created directory at: artifacts]
[2025-02-24 23:39:22,319: INFO: main_utils: created directory at: artifacts/data_ingestion]
[2025-02-24 23:39:24,136: INFO: 4156189403: artifacts/data_ingestion/HIV.csv downloaded with folowing info: 
x-amz-id-2: QO72qam4WSQSO4T5v3mN4MQEBGfR1NCy8lCi9JZE7XLsouRbOFsYwrqyGRwzRg6J/DLEACoTid9jOGU7LDcaLJ7qingiGUR3n7by1f3Xt8A=
x-amz-request-id: JKTQ0BMK2WZA77BE
Date: Mon, 24 Feb 2025 22:39:23 GMT
Last-Modified: Fri, 10 Jul 2020 06:45:51 GMT
ETag: "9ad10c88f82f1dac7eb5c52b668c30a7"
Accept-Ranges: bytes
Content-Type: text/csv
Content-Length: 2193844
Server: AmazonS3
Connection: close

]
